In [32]:
#libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import folium
import os
import sys
sys.path.append(os.path.abspath('abstract_flows'))
import arrow
import grid
import flows

In [33]:
%%script false --no-raise-error
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150
%config InLineBackend.figure_format = 'retina'
print('Retinas activated')

In [34]:
dir = '/home/user/Desktop/Datos/'
#dir = '/Users/antoniomendez/Desktop/Tesis/Datos/datos_limpios/'

In [35]:
estaciones_eco = np.load(dir + 'Adj_eco/matrices_estaciones/est_2019.npy')
#estaciones_eco = np.load('/Users/antoniomendez/Desktop/Tesis/Datos/Adj_eco/matrices_estaciones/est_2019.npy')

In [36]:
data_eco = pd.read_csv(dir + 'ecobici/ecobici_2019.csv')
data_eco.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Tiempo_viaje
0,M,36,11444,67,2019-01-01,0:08:44,36,2019-01-01,0:14:03,5.316667
1,M,36,9196,55,2019-01-01,0:12:05,136,2019-01-01,0:42:11,30.100000
2,M,38,2612,29,2019-01-01,0:13:15,115,2019-01-01,0:37:47,24.533333
3,M,43,7463,29,2019-01-01,0:13:36,115,2019-01-01,0:38:04,24.466667
4,M,37,3987,136,2019-01-01,0:15:30,35,2019-01-01,0:22:47,7.283333


In [38]:
%%script false --no-raise-error
# we only conserve the data of the third week of the year
data_eco = data_eco[(data_eco['Fecha_Retiro'] >= '2019-01-14') & (data_eco['Fecha_Retiro'] <= '2019-01-20')]
data_eco.head()

In [39]:
data_eco.tail()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Tiempo_viaje
8382173,F,27,11758,420,2019-12-30,23:57:15,427,2019-12-31,0:06:07,8.866667
8382174,M,29,6976,420,2019-12-30,23:57:26,427,2019-12-31,0:05:59,8.550000
8382175,M,23,6778,58,2019-12-30,23:59:01,36,2019-12-31,0:02:20,3.316667
8382176,F,33,4048,99,2019-12-30,23:59:15,148,2019-12-31,0:11:32,12.283333
8382177,M,34,9987,99,2019-12-30,23:59:51,148,2019-12-31,0:11:42,11.850000


In [40]:
hours = [f'{i}:00:00' for i in range(1, 24)] + ['0:00:00']

In [41]:
dates = data_eco['Fecha_Retiro'].unique()
dates = [date for date in dates if date >= '2019-01-01' and date <= '2019-12-31']
dates.sort()

In [42]:
def create_adjacency_matrix(flows_df, dims, directed=True):
    '''
    Parameters:
        flows_df (DataFrame): DataFrame with columns i_A, j_A, i_B, j_B, flow_count.
        dims (tuple): Dimensions of the grid (rows, columns).
        directed (bool): If True, the graph is directed; if False, the graph is undirected.

    Returns:
        adjacency_matrix (numpy.ndarray): Adjacency matrix of the graph with size (dims[0]*dims[1], (dims[0]*dims[1])).
        nodes (list): List of nodes (i, j) in the same order as the adjacency matrix.
    '''
    # Initialize the adjacency matrix with zeros
    n = dims[0] * dims[1]
    adjacency_matrix = np.zeros((n, n), dtype=int)
    
    # Create a mapping from (i, j) to a unique index
    def node_to_index(i, j):
        return i * dims[1] + j
    
    # Fill the adjacency matrix
    for _, row in flows_df.iterrows():
        i_A, j_A = row['i_A'], row['j_A']
        i_B, j_B = row['i_B'], row['j_B']
        flow_count = row['flow_count']
        
        # Convert (i, j) to indices
        idx_A = node_to_index(i_A, j_A)
        idx_B = node_to_index(i_B, j_B)
        
        # Update the adjacency matrix
        adjacency_matrix[idx_A, idx_B] += flow_count
        
        if not directed:
            adjacency_matrix[idx_B, idx_A] += flow_count
    
    # Generate the list of nodes in the same order as the adjacency matrix
    nodes = [(i, j) for i in range(dims[0]) for j in range(dims[1])]
    
    return adjacency_matrix, nodes

In [43]:
def compute_laplacian(adjacency_matrix, normalized=False):
    '''
    Parameters:
        adjacency_matrix (numpy.ndarray): The adjacency matrix of the graph.
        normalized (bool): If True, compute the normalized Laplacian. Default is False.

    Returns:
        laplacian (numpy.ndarray): The Laplacian matrix.
    '''
    # Compute the degree matrix
    degree_matrix = np.diag(np.sum(adjacency_matrix, axis=1))
    
    # Compute the Laplacian matrix
    laplacian = degree_matrix - adjacency_matrix
    
    if normalized:
        # Compute the normalized Laplacian
        degree_matrix_inv_sqrt = np.linalg.inv(np.sqrt(degree_matrix))
        laplacian = np.eye(degree_matrix.shape[0]) - degree_matrix_inv_sqrt @ adjacency_matrix @ degree_matrix_inv_sqrt
    
    return laplacian

In [44]:
def date_to_custom_integer(date_str):
    '''
    Convert a date string in 'yyyy-mm-dd H:MM:SS' format to a custom integer representation (yyyymmddHHMMSS).
    
    Parameters:
        date_str (str): Date and time string in 'yyyy-mm-dd H:MM:SS' format.
    
    Returns:
        int: Integer representation of the date and time in 'yyyymmddHHMMSS' format.
    '''
    # Parse the date string into a datetime object
    dt = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')
    
    # Format the datetime object into 'yyyymmddHHMMSS' and convert to integer
    custom_integer = int(dt.strftime('%Y%m%d%H%M%S'))
    
    return custom_integer

In [45]:
matrices = []
nodes = []
laplacians = []
dates_considered = []

In [46]:
# it will take like a minute per date
for date in dates:
    print('Processing: ', date)
    for hour in hours:
        if hour == '0:00:00':
            current_data = data_eco[(data_eco['Fecha_Retiro'] == date) & (data_eco['Hora_Retiro'] >= '23:00:00')]
        else:
            current_data = data_eco[(data_eco['Fecha_Retiro'] == date) & (data_eco['Hora_Retiro'] <= hour)]
        if current_data.empty:
            continue
        current_counter = flows.count_trips_ecobici(current_data, threshold=1, directed=True)
        current_stations = pd.unique(np.concatenate((current_counter['Est_A'].unique(), current_counter['Est_B'].unique())))
        current_grid = grid.Grid(9, 5, 'ecobici')
        current_station_cells = flows.stations_and_cells(current_grid.geodataframe(), current_stations, estaciones_eco)
        current_graph_df = flows.abstract_flows(current_counter, current_grid.geodataframe(), current_station_cells, estaciones_eco, threshold=1)
        current_matrix, current_nodes = create_adjacency_matrix(current_graph_df, [9,5], directed=True)
        #current_laplacian = compute_laplacian(current_matrix, normalized=False)
        matrices.append(current_matrix)
        nodes.append(current_nodes)
        #laplacians.append(current_laplacian)
        dates_considered.append(date + ' ' + hour)

Processing:  2019-01-01
Processing:  2019-01-02
Processing:  2019-01-03
Processing:  2019-01-04
Processing:  2019-01-05
Processing:  2019-01-06
Processing:  2019-01-07
Processing:  2019-01-08
Processing:  2019-01-09
Processing:  2019-01-10
Processing:  2019-01-11
Processing:  2019-01-12
Processing:  2019-01-13
Processing:  2019-01-14
Processing:  2019-01-15
Processing:  2019-01-16
Processing:  2019-01-17
Processing:  2019-01-18
Processing:  2019-01-19
Processing:  2019-01-20
Processing:  2019-01-21
Processing:  2019-01-22
Processing:  2019-01-23
Processing:  2019-01-24
Processing:  2019-01-25
Processing:  2019-01-26
Processing:  2019-01-27
Processing:  2019-01-28
Processing:  2019-01-29
Processing:  2019-01-30
Processing:  2019-01-31
Processing:  2019-02-01
Processing:  2019-02-02
Processing:  2019-02-03
Processing:  2019-02-04
Processing:  2019-02-05
Processing:  2019-02-06
Processing:  2019-02-07


KeyboardInterrupt: 

In [ ]:
dates_considered = [date_to_custom_integer(date) for date in dates_considered]

In [ ]:
np.save('vector_files/adjacency_matrices_2019.npy', matrices)
np.save('vector_files/nodes_2019.npy', nodes)
#np.save('vector_files/laplacians_2019.npy', laplacians)
np.save('vector_files/dates_considered_2019.npy', dates_considered)

In [ ]:
matrices = np.array(matrices)
matrices.shape

(168, 45, 45)

In [ ]:
def getcompmatrix():
	holiday = [i for i in range(1506765600, 1507478400+3600, 3600)]
	weekd = [1,2,3,4,5]
	noweekd = [6,0]
	listtime = np.load('vector_files/dates_considered.npy')
	
	
	co = np.zeros((listtime.shape[0],listtime.shape[0]))
	print(co.shape)
	for i in range(co.shape[0]):
		tod = int(((listtime[i]-1427644800)/3600/24+1)%7)
		print(tod)
		for j in range(co.shape[1]):
			tod2 = int(((listtime[j]-1427644800)/3600/24+1)%7)
			if (tod in weekd) & (tod2 in weekd) & (abs(i-j)%24==0) & (listtime[i] not in holiday) & (listtime[j] not in holiday):
				co[i][j] = 1
			if ((tod in noweekd) | (listtime[i] in holiday)) & ((tod2 in noweekd) | (listtime[j] in holiday)) & (abs(i-j)%24==0):
				co[i][j] = 1
	for i in range(co.shape[0]):
		print(co[i].sum())
	print(co.sum())
	np.save("vector_files/company_matrix_2019.npy",co)
	
getcompmatrix()

(168, 168)
4
4
4
4
4
5
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
6
6
4
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
4
4
4
4
4
1
6
6
6
6
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
2
2
6
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
6
6
6
6
6
6
6
4
1
1
1
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
4
4
4
1
6
6
6
6
6
6
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
6
3
4
4
4
4
4
4
4
4
4
5
5
5
5
5
5
5
5
5
6
6
6
6
3
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
6.0
2.0
1.0
2.0
2.0
2.0
3.0
3.0
3.0
3.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
6.0
5.0
6.0
5.0
5.0
5.0
4.0
4.0
4.0
4.0
5.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
6.0
5.0
6.0
5.0
5.0
5.0
4.0
4.0
4.0
4.0
2.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
6.0
5.0
6.0
2.0
2.0
2.0
3.0
3.0
3.0
3.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
6.0
5.0
6.0
5.0
5.0
5.0
4.0
4.0
4.0
4.0
5.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
1.0
2.0
6.0
5.0
5.0
5.0
4.0
4.0
4.0
4.0
2.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
5.0
6.0
5.0
6.0
5.0
5.0
5.0
3